In [1]:
import pandas as pd
import numpy as np

In [2]:
import gremlin_python
from gremlin_python import statics
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier
from gremlin_python.process.traversal import Bindings
from gremlin_python.process.traversal import WithOptions
from gremlin_python.structure.graph import Graph
from gremlin_python.driver import client, serializer, protocol

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
db = pd.read_csv("db.csv")

In [5]:
db = db[db["RevSumDivisionName"]=="Azure Standalone"]

In [6]:
db = db[["OrgName", "CustomerID","CountryName", "SalesMonthID","PricingLevelName", "ProductFamilyName", "ActualLicenseCnt", "ActualRevenueAmt"]]

In [7]:
db = db.sort_values(["OrgName","CustomerID" ,"CountryName","SalesMonthID"])

In [8]:
short_db = db.head(1000000).copy()

In [9]:
short_db = short_db.groupby(["OrgName","CustomerID", "CountryName","SalesMonthID", "PricingLevelName", "ProductFamilyName"]).agg({'ActualRevenueAmt':'sum','ActualLicenseCnt':'sum'}).reset_index()

In [143]:
db = short_db[(short_db["CustomerID"]== 86952148) | (short_db["CustomerID"]== 83445107)| (short_db["CustomerID"]== 47175135)]

In [145]:
db.reset_index(drop=True, inplace=True)
db= db.drop_duplicates()

In [148]:
db["MonthSinceMicrosoft"]=0

for org in db["OrgName"].unique():

    first_month = db[db["OrgName"]==org].iloc[0]["SalesMonthID"].copy()

    org_selected = db[db["OrgName"]==org].copy()

    org_selected["MonthSinceMicrosoft"] = org_selected["SalesMonthID"] - first_month 

    db[db["OrgName"]==org] = org_selected


In [4]:
client = client.Client('wss://jtacosmos.gremlin.cosmosdb.azure.com:443/','g', 
            username="/dbs/jta_graphdb/colls/graph_collection", 
            password="1MiVl7ZaA8xpILwTFJCFN07Bjjd3Ln5cpjEXjQi46E3L5qA0sFoVhjc836OP0oQA31PRjQBqe9SvPymgm2teTA==",
            message_serializer= serializer.GraphSONSerializersV2d0())

In [152]:
def cleanup_graph(gremlin_client):
    callback = gremlin_client.submitAsync("g.V().drop()")
    if callback.result() is not None:
        print("Cleaned up the graph!")

def insert_vertices(gremlin_client):
    for vertex in vertices:
        callback = gremlin_client.submitAsync(vertex)
        if callback.result() is not None:
            print("Inserted this vertex:\n{0}".format(callback.result().one()))
        else:
            print("Something went wrong with this query: {0}".format(vertex))

def insert_edges(gremlin_client):
    for edge in edges:
        callback = gremlin_client.submitAsync(edge)
        if callback.result() is not None:
            print("Inserted this edge:\n{0}".format(callback.result().one()))
        else:
            print("Something went wrong with this query:\n{0}".format(edge))

def handler():
    # Initialise client
    print('Initialising client...')
    gremlin_client = client
    print('Client initialised!')

    # Purge graph
    cleanup_graph(gremlin_client)

    # Insert vertices (i.e. nodes)
    insert_vertices(gremlin_client)

    # Insert edges (i.e. nodes)
    insert_edges(gremlin_client)

    print('Finished!')

In [157]:
def define_structure(db):

    company = None
    category = None
    vertices = []
    edges =[]
    categories=[]
    old_categories =[]
    idex = []
    old_idex = []

    for i in range(db.shape[0]):
    
        if company == None:
            company = db["CustomerID"].values[i]
            counter = 0
            

        else: 
            if company != db["CustomerID"].values[i]:
                company = db["CustomerID"].values[i]
                old_idex = []
                old_categories =[]
                counter =0

        if counter == 0:
            vert =  "g.addV('Company').property('id','" + db["OrgName"].values[i]  +"').property('name', '" + db["OrgName"].values[i]  +"').property('country',  '" + db["CountryName"].values[i]  +"')"
            vertices.append(vert)

        else: 
            msm = db["MonthSinceMicrosoft"].values[i]
            month = db["SalesMonthID"].values[i]
            level = db["PricingLevelName"].values[i]
            category = db["ProductFamilyName"].values[i]

           
            categories.append(category)

            vert =  "g.addV('Contract').property('id','" + str(category) + "_" +  str(company) + "_" + str(month) + "_" + str(level) +"').property('name', '" + str(category) + "_" +  str(company) + "_" + str(month) +  "_" + str(level) +"').property('product', '" + str(category)  +"').property('pricing_level',  '" + str(level)  + "').property('months_with_microsoft',  '" + str(msm)  + "')"

            idex.append(str(category) + "_" + str(company) + "_" + str(month)+ "_" + str(level))
            vertices.append(vert)

        
            if ((i == db.shape[0]-1 ) or (month != db["SalesMonthID"].values[i+1]) | (company != db["CustomerID"].values[i+1])):
            
                if len(old_idex) == 0:

                    for j, idx in enumerate(idex):

                        t = list(range(len(idex)))[::-1]
                        t = t[j]
                        edg = "g.V('" + db["OrgName"].values[i]  + "').addE('purchased').to(g.V('"+ idx + "')).property('licenses',  %f).property('revenue',  %f)" %(db["ActualLicenseCnt"].values[i-t], db["ActualRevenueAmt"].values[i-t]) 
                        edges.append(edg) 
            
                else:

                    for j, idx in enumerate(idex):

                        t = list(range(len(idex)))[::-1]
                        t = t[j]

                        cat = idx.split("_")[0]
                
                        if cat not in old_categories:

                            for old_idx in old_idex:

                                edg = "g.V('"+ old_idx + "').addE('related').to(g.V('" + idx + "')).property('licenses',  %f).property('revenue',  %f)" %(db["ActualLicenseCnt"].values[i-t], db["ActualRevenueAmt"].values[i-t]) 
                                edges.append(edg)  
                            
                        else:
                        
                            for old_idx in old_idex:
                            
                                old_cat = old_idx.split("_")[0]
                            
                                if old_cat == cat:
                                    edg = "g.V('" + old_idx + "').addE('related').to(g.V('" + idx + "')).property('licenses',  %f).property('revenue',  %f)" %(db["ActualLicenseCnt"].values[i-t], db["ActualRevenueAmt"].values[i-t]) 
                                    edges.append(edg) 
                                


                old_categories = categories  
                old_idex = idex
                
                categories=[]
                idex = []
     
    
        counter = 1
        
    db["ProductFamilyName"]= db["ProductFamilyName"].str.replace("/", "-")

    return(db, vertices, edges)

In [158]:
db, vertices, edges = define_structure(db)

In [159]:
handler()

Initialising client...
Client initialised!
Cleaned up the graph!
Inserted this vertex:
[{'id': 'ANDREW thomas', 'label': 'Company', 'type': 'vertex', 'properties': {'name': [{'id': 'ANDREW thomas|name', 'value': 'ANDREW thomas'}], 'country': [{'id': '67e46d00-ff40-4383-b02b-2f04dd5eb48b', 'value': 'Canada'}]}}]
Inserted this vertex:
[{'id': 'Azure Consumption_47175135_325_Hosting', 'label': 'Contract', 'type': 'vertex', 'properties': {'name': [{'id': 'Azure Consumption_47175135_325_Hosting|name', 'value': 'Azure Consumption_47175135_325_Hosting'}], 'product': [{'id': '54d6555d-133b-44ff-9ff3-84c0b70a1fe4', 'value': 'Azure Consumption'}], 'pricing_level': [{'id': '7f36b1c4-82cf-456b-aa8b-5a0edc4fa85b', 'value': 'Hosting'}], 'months_with_microsoft': [{'id': 'd2cac7ed-c84c-41d2-9ef8-61870e696c7d', 'value': '0'}]}}]
Inserted this vertex:
[{'id': 'Azure Monetary Credit Promo_47175135_325_Hosting', 'label': 'Contract', 'type': 'vertex', 'properties': {'name': [{'id': 'Azure Monetary Credit P

In [2]:
client.close()